In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
DATA_PATH = "/content/drive/MyDrive/the_datas/data/"
DATA_PATH

'/content/drive/MyDrive/the_datas/data/'

In [ ]:
SEED = 42

In [ ]:
import pandas as pd
import numpy as np

train_tr = pd.read_csv(f"{DATA_PATH}store_train_transactions.csv") # 학습용 구매기록 데이터
train_target = pd.read_csv(f"{DATA_PATH}store_train.csv") # 학습용 정답 데이터
test_tr = pd.read_csv(f"{DATA_PATH}store_test_transactions.csv") # 테스트용 구매기록 데이터
submit = pd.read_csv(f"{DATA_PATH}store_submission.csv") # 제출 양식 데이터

train_tr.shape , train_target.shape , test_tr.shape , submit.shape

((523105, 7), (14940, 2), (441196, 7), (12225, 2))

In [ ]:

train_ft = pd.read_csv(f"{DATA_PATH}train_common_1_완료.csv") # 학습 데이터(피처)
test_ft = pd.read_csv(f"{DATA_PATH}test_common_1_완료.csv") # 테스트 데이터(피처)

train_ft.shape , test_ft.shape

((14940, 356), (12225, 356))

# 특성 생성 실험

0.7130706978777662

In [ ]:
train_tr["구매일시"] = pd.to_datetime(train_tr["구매일시"])
test_tr["구매일시"] = pd.to_datetime(test_tr["구매일시"])

In [ ]:
train_tr["구매일시"].min(), train_tr["구매일시"].max()

(Timestamp('2004-05-01 09:40:00'), Timestamp('2005-04-29 20:03:00'))

In [ ]:
# 1월부터 12월까지의 구매 비율 추가
agg_list = []
agg_list += [(f'{n}월_구매비율', lambda x: np.mean(x.dt.month == n)) for n in range(1, 13)]

tmp = train_tr.groupby('ID')["구매일시"].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left',on="ID")
train_ft.head()

tmp = test_tr.groupby('ID')["구매일시"].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left',on="ID")
test_ft.head()

,ID,구매횟수,내점일수,구매주기,주말방문비율,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,...,3월_구매비율,4월_구매비율,5월_구매비율,6월_구매비율,7월_구매비율,8월_구매비율,9월_구매비율,10월_구매비율,11월_구매비율,12월_구매비율
0,test_0,7,5,32,0.571429,0.285714,0.285714,0.428571,0.000000,1,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
1,test_1,4,3,54,0.250000,0.000000,0.000000,0.750000,0.250000,0,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
2,test_2,4,2,8,0.000000,0.000000,0.600000,0.400000,0.000000,0,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
3,test_3,28,16,16,0.031250,0.062500,0.468750,0.343750,0.125000,3,...,0.09375,0.09375,0.09375,0.09375,0.09375,0.09375,0.09375,0.09375,0.09375,0.09375
4,test_4,19,9,29,0.476190,0.380952,0.285714,0.238095,0.095238,6,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000


In [ ]:

#대분류, 구매가격 pivot 추가

train_tmp = pd.pivot_table(train_tr,index="ID",columns="대분류",values="구매가격",aggfunc="count",fill_value=0).add_prefix("대_pivot_cnt").reset_index()
train_ft = train_ft.merge(train_tmp,how="left",on="ID")
train_ft.head()
test_tmp = pd.pivot_table(test_tr,index="ID",columns="대분류",values="구매가격",aggfunc="count",fill_value=0).add_prefix("대_pivot_cnt").reset_index()
test_tmp.head()

for col in train_tmp.columns:
    if col not in test_tmp.columns:
        test_tmp[col] = 0

test_tmp = test_tmp[train_tmp.columns]

test_ft = test_ft.merge(test_tmp,how="left",on="ID")
test_ft.head()

,ID,구매횟수,내점일수,구매주기,주말방문비율,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,...,대_pivot_cnt여성캐주얼,대_pivot_cnt여성캐쥬얼,대_pivot_cnt영라이브,대_pivot_cnt영어덜트캐쥬얼,대_pivot_cnt영캐릭터,대_pivot_cnt영플라자,대_pivot_cnt잡화,대_pivot_cnt잡화파트,대_pivot_cnt케주얼_구두_아동,대_pivot_cnt패션잡화
0,test_0,7,5,32,0.571429,0.285714,0.285714,0.428571,0.000000,1,...,0,1,0,0,0,0,1,0,0,0
1,test_1,4,3,54,0.250000,0.000000,0.000000,0.750000,0.250000,0,...,1,0,0,0,0,0,0,0,0,0
2,test_2,4,2,8,0.000000,0.000000,0.600000,0.400000,0.000000,0,...,0,0,0,0,0,0,0,2,2,0
3,test_3,28,16,16,0.031250,0.062500,0.468750,0.343750,0.125000,3,...,0,0,0,0,2,15,0,0,0,13
4,test_4,19,9,29,0.476190,0.380952,0.285714,0.238095,0.095238,6,...,9,0,0,0,0,1,0,0,5,1


0.7107565592454816

In [ ]:
q25, q50, q75 = train_ft['최소구매액'].quantile([0.25, 0.5, 0.75])

train_ft['최소구매액_bin'] = pd.cut(train_ft['최소구매액'], bins=[0, q25, q50, q75, np.inf], labels=[1, 2, 3, 4])
test_ft['최소구매액_bin'] = pd.cut(test_ft['최소구매액'], bins=[0, q25, q50, q75, np.inf], labels=[1, 2, 3, 4])

train_ft.shape, test_ft.shape

((14940, 397), (12225, 397))

In [ ]:
q25, q50, q75 = train_ft['중분류_nunique'].quantile([0.25, 0.5, 0.75])
train_ft['중분류_nunique_bin'] = pd.cut(train_ft['중분류_nunique'], bins=[0, q25, q50, q75, np.inf], labels=[1, 2, 3, 4])
test_ft['중분류_nunique_bin'] = pd.cut(test_ft['중분류_nunique'], bins=[0, q25, q50, q75, np.inf], labels=[1, 2, 3, 4])

In [ ]:
for i in range(1,5):
    for j in range(1,5):
        train_ft[f'최소구매액_중분류수_{i}_{j}'] = ((train_ft['최소구매액_bin'] == i) & (train_ft['중분류_nunique_bin'] == j)).astype(int)
        test_ft[f'최소구매액_중분류수_{i}_{j}'] = ((test_ft['최소구매액_bin'] == i) & (test_ft['중분류_nunique_bin'] == j)).astype(int)

0.7154512315623972

In [ ]:
Q1 = train_tr['구매가격'].quantile(0.25)
Q3 = train_tr['구매가격'].quantile(0.75)
IQR = Q3 - Q1

In [ ]:
Q3, Q1

(119700.0, 22520.0)

In [ ]:
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

In [ ]:
lower_bound, upper_bound

(-123250.0, 265470.0)

In [ ]:
train_ft['구매가격_특이값'] = ((train_ft['총구매액'] > upper_bound)).astype(int)
test_ft['구매가격_특이값'] = ((test_ft['총구매액'] > upper_bound)).astype(int) #(test_ft['총구매액'] < lower_bound) |

0.714270454857448

횟수까지 추가해서 가격/중분류 네개 다 추가
0.712716483135802

In [ ]:
purchase_counts = train_tr.groupby('ID')['구매가격'].count().reset_index()
purchase_counts.columns = ['ID', '구매횟수']

In [ ]:
같은가격재구매 = train_tr[train_tr.duplicated(subset=['ID', '구매가격'], keep=False)]
같은가격재구매횟수 = 같은가격재구매.groupby('ID')['구매가격'].count().reset_index()
같은가격재구매횟수.columns = ['ID', '같은가격재구매횟수']

In [ ]:
purchase_counts = purchase_counts.merge(같은가격재구매횟수, on='ID', how='left').fillna(0)
purchase_counts['반복구매비율'] = purchase_counts['같은가격재구매횟수'] / purchase_counts['구매횟수']


In [ ]:
train_ft = train_ft.merge(purchase_counts[['ID', '반복구매비율']], on='ID', how='left')

In [ ]:
train_ft.head()

,ID,구매횟수,내점일수,구매주기,주말방문비율,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,...,최소구매액_중분류수_3_1,최소구매액_중분류수_3_2,최소구매액_중분류수_3_3,최소구매액_중분류수_3_4,최소구매액_중분류수_4_1,최소구매액_중분류수_4_2,최소구매액_중분류수_4_3,최소구매액_중분류수_4_4,구매가격_특이값,반복구매비율
0,train_0,16,11,23,0.250000,0.050000,0.250000,0.400000,0.300000,3,...,0,1,0,0,0,0,0,0,1,0.100000
1,train_1,41,21,16,0.023810,0.357143,0.166667,0.357143,0.119048,3,...,0,0,0,0,0,0,0,0,1,0.119048
2,train_2,102,56,6,0.210526,0.464912,0.140351,0.175439,0.219298,0,...,0,0,0,0,0,0,0,0,1,0.210526
3,train_3,191,92,3,0.189573,0.379147,0.180095,0.236967,0.203791,3,...,0,0,0,0,0,0,0,0,1,0.473934
4,train_4,55,27,11,0.258065,0.112903,0.612903,0.209677,0.064516,4,...,0,0,0,0,0,0,0,0,1,0.064516


In [ ]:
purchase_counts = train_tr.groupby('ID')['구매가격'].count().reset_index()
purchase_counts.columns = ['ID', '구매횟수']
같은가격재구매 = train_tr[train_tr.duplicated(subset=['ID', '구매가격'], keep=False)]
같은가격재구매횟수 = 같은가격재구매.groupby('ID')['구매가격'].count().reset_index()
같은가격재구매횟수.columns = ['ID', '같은가격재구매횟수']
purchase_counts = purchase_counts.merge(같은가격재구매횟수, on='ID', how='left').fillna(0)
purchase_counts['반복구매비율'] = purchase_counts['같은가격재구매횟수'] / purchase_counts['구매횟수']
train_ft = train_ft.merge(purchase_counts[['ID', '같은가격재구매횟수','반복구매비율']], on='ID', how='left')


In [ ]:
purchase_counts = test_tr.groupby('ID')['구매가격'].count().reset_index()
purchase_counts.columns = ['ID', '구매횟수']
같은가격재구매 = test_tr[test_tr.duplicated(subset=['ID', '구매가격'], keep=False)]
같은가격재구매횟수 = 같은가격재구매.groupby('ID')['구매가격'].count().reset_index()
같은가격재구매횟수.columns = ['ID', '같은가격재구매횟수']
purchase_counts = purchase_counts.merge(같은가격재구매횟수, on='ID', how='left').fillna(0)
purchase_counts['반복구매비율'] = purchase_counts['같은가격재구매횟수'] / purchase_counts['구매횟수']
test_ft = test_ft.merge(purchase_counts[['ID', '반복구매비율']], on='ID', how='left')

In [ ]:
purchase_counts = test_tr.groupby('ID')['구매가격'].count().reset_index()
purchase_counts.columns = ['ID', '구매횟수']
같은가격재구매 = test_tr[test_tr.duplicated(subset=['ID', '구매가격'], keep=False)]
같은가격재구매횟수 = 같은가격재구매.groupby('ID')['구매가격'].count().reset_index()
같은가격재구매횟수.columns = ['ID', '같은가격재구매횟수']
purchase_counts = purchase_counts.merge(같은가격재구매횟수, on='ID', how='left').fillna(0)
purchase_counts['반복구매비율'] = purchase_counts['같은가격재구매횟수'] / purchase_counts['구매횟수']
test_ft = test_ft.merge(purchase_counts[['ID', '같은가격재구매횟수', '반복구매비율']], on='ID', how='left')

In [ ]:
test_ft.head()

,ID,구매횟수,내점일수,구매주기,주말방문비율,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,...,최소구매액_중분류수_3_3,최소구매액_중분류수_3_4,최소구매액_중분류수_4_1,최소구매액_중분류수_4_2,최소구매액_중분류수_4_3,최소구매액_중분류수_4_4,구매가격_특이값,반복구매비율_x,같은가격재구매횟수,반복구매비율_y
0,test_0,7,5,32,0.571429,0.285714,0.285714,0.428571,0.000000,1,...,0,0,1,0,0,0,1,0.000,0.0,0.000
1,test_1,4,3,54,0.250000,0.000000,0.000000,0.750000,0.250000,0,...,0,0,1,0,0,0,1,0.000,0.0,0.000
2,test_2,4,2,8,0.000000,0.000000,0.600000,0.400000,0.000000,0,...,0,0,1,0,0,0,1,0.000,0.0,0.000
3,test_3,28,16,16,0.031250,0.062500,0.468750,0.343750,0.125000,3,...,0,0,0,0,0,0,1,0.125,4.0,0.125
4,test_4,19,9,29,0.476190,0.380952,0.285714,0.238095,0.095238,6,...,0,0,0,0,0,0,1,0.000,0.0,0.000


0.7143624217714903

In [ ]:
purchase_counts = train_tr.groupby('ID')['중분류'].count().reset_index()
purchase_counts.columns = ['ID', '중분류수']
같은중분류재구매 = train_tr[train_tr.duplicated(subset=['ID', '중분류'], keep=False)]
같은중분류재구매횟수 = 같은중분류재구매.groupby('ID')['중분류'].count().reset_index()
같은중분류재구매횟수.columns = ['ID', '같은중분류재구매횟수']
purchase_counts = purchase_counts.merge(같은중분류재구매횟수, on='ID', how='left').fillna(0)
purchase_counts['반복중분류비율'] = purchase_counts['같은중분류재구매횟수'] / purchase_counts['중분류수']
train_ft = train_ft.merge(purchase_counts[['ID', '반복중분류비율']], on='ID', how='left')

In [ ]:
purchase_counts = train_tr.groupby('ID')['중분류'].count().reset_index()
purchase_counts.columns = ['ID', '중분류수']
같은중분류재구매 = train_tr[train_tr.duplicated(subset=['ID', '중분류'], keep=False)]
같은중분류재구매횟수 = 같은중분류재구매.groupby('ID')['중분류'].count().reset_index()
같은중분류재구매횟수.columns = ['ID', '같은중분류재구매횟수']
purchase_counts = purchase_counts.merge(같은중분류재구매횟수, on='ID', how='left').fillna(0)
purchase_counts['반복중분류비율'] = purchase_counts['같은중분류재구매횟수'] / purchase_counts['중분류수']
train_ft = train_ft.merge(purchase_counts[['ID', '같은중분류재구매횟수', '반복중분류비율']], on='ID', how='left')


In [ ]:
train_ft.head()

,ID,구매횟수,내점일수,구매주기,주말방문비율,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,...,최소구매액_중분류수_4_2,최소구매액_중분류수_4_3,최소구매액_중분류수_4_4,구매가격_특이값,반복구매비율_x,같은가격재구매횟수,반복구매비율_y,반복중분류비율_x,같은중분류재구매횟수,반복중분류비율_y
0,train_0,16,11,23,0.250000,0.050000,0.250000,0.400000,0.300000,3,...,0,0,0,1,0.100000,2.0,0.100000,0.550000,11.0,0.550000
1,train_1,41,21,16,0.023810,0.357143,0.166667,0.357143,0.119048,3,...,0,0,0,1,0.119048,5.0,0.119048,0.642857,27.0,0.642857
2,train_2,102,56,6,0.210526,0.464912,0.140351,0.175439,0.219298,0,...,0,0,0,1,0.210526,24.0,0.210526,0.859649,98.0,0.859649
3,train_3,191,92,3,0.189573,0.379147,0.180095,0.236967,0.203791,3,...,0,0,0,1,0.473934,100.0,0.473934,0.857820,181.0,0.857820
4,train_4,55,27,11,0.258065,0.112903,0.612903,0.209677,0.064516,4,...,0,0,0,1,0.064516,4.0,0.064516,0.822581,51.0,0.822581


In [ ]:
purchase_counts = test_tr.groupby('ID')['중분류'].count().reset_index()
purchase_counts.columns = ['ID', '중분류수']
같은중분류재구매 = test_tr[test_tr.duplicated(subset=['ID', '중분류'], keep=False)]
같은중분류재구매횟수 = 같은중분류재구매.groupby('ID')['중분류'].count().reset_index()
같은중분류재구매횟수.columns = ['ID', '같은중분류재구매횟수']
purchase_counts = purchase_counts.merge(같은중분류재구매횟수, on='ID', how='left').fillna(0)
purchase_counts['반복중분류비율'] = purchase_counts['같은중분류재구매횟수'] / purchase_counts['중분류수']
test_ft = test_ft.merge(purchase_counts[['ID', '반복중분류비율']], on='ID', how='left')

In [ ]:
purchase_counts = test_tr.groupby('ID')['중분류'].count().reset_index()
purchase_counts.columns = ['ID', '중분류수']
같은중분류재구매 = test_tr[test_tr.duplicated(subset=['ID', '중분류'], keep=False)]
같은중분류재구매횟수 = 같은중분류재구매.groupby('ID')['중분류'].count().reset_index()
같은중분류재구매횟수.columns = ['ID', '같은중분류재구매횟수']
purchase_counts = purchase_counts.merge(같은중분류재구매횟수, on='ID', how='left').fillna(0)
purchase_counts['반복중분류비율'] = purchase_counts['같은중분류재구매횟수'] / purchase_counts['중분류수']
test_ft = test_ft.merge(purchase_counts[['ID', '같은중분류재구매횟수', '반복중분류비율']], on='ID', how='left')

0.7156154989770538(여기서부터 희원님 괄호 수정 기준)

In [ ]:
agg_list = [
             ('주구매_브랜드코드', lambda x:x.mode()[0]),
             ]

tmp = train_tr.groupby('ID')["브랜드코드"].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left',on="ID")
train_ft.head()

tmp = test_tr.groupby('ID')["브랜드코드"].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left',on="ID")
test_ft.head()

,ID,구매횟수,내점일수,구매주기,주말방문비율,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,...,최소구매액_중분류수_4_3,최소구매액_중분류수_4_4,구매가격_특이값,반복구매비율_x,같은가격재구매횟수,반복구매비율_y,반복중분류비율_x,같은중분류재구매횟수,반복중분류비율_y,주구매_브랜드코드
0,test_0,7,5,32,0.571429,0.285714,0.285714,0.428571,0.000000,1,...,0,0,1,0.000,0.0,0.000,0.714286,5.0,0.714286,5100
1,test_1,4,3,54,0.250000,0.000000,0.000000,0.750000,0.250000,0,...,0,0,1,0.000,0.0,0.000,0.000000,0.0,0.000000,5149
2,test_2,4,2,8,0.000000,0.000000,0.600000,0.400000,0.000000,0,...,0,0,1,0.000,0.0,0.000,0.400000,2.0,0.400000,5111
3,test_3,28,16,16,0.031250,0.062500,0.468750,0.343750,0.125000,3,...,0,0,1,0.125,4.0,0.125,0.875000,28.0,0.875000,5111
4,test_4,19,9,29,0.476190,0.380952,0.285714,0.238095,0.095238,6,...,0,0,1,0.000,0.0,0.000,0.666667,14.0,0.666667,5204


In [ ]:
#요일별 구매비율

In [ ]:
train_tr["구매일시"] = pd.to_datetime(train_tr["구매일시"])
test_tr["구매일시"] = pd.to_datetime(test_tr["구매일시"])

In [ ]:
agg_list = [
    ('월요일방문비율', lambda x : np.mean(x.dt.weekday == 0)),
    ('화요일방문비율', lambda x : np.mean(x.dt.weekday == 1)),
    ('수요일방문비율', lambda x : np.mean(x.dt.weekday == 2)),
    ('목요일방문비율', lambda x : np.mean(x.dt.weekday == 3)),
    ('금요일방문비율', lambda x : np.mean(x.dt.weekday == 4)),
    ('토요일방문비율', lambda x : np.mean(x.dt.weekday == 5)),
    ('일요일방문비율', lambda x : np.mean(x.dt.weekday == 6)),
    #('금토일방문비율', lambda x : np.mean(x.dt.weekday >= 4)),
    #('금오후토일방문비율', lambda x: np.mean((x.dt.weekday >= 5) | ((x.dt.weekday == 4) & (x.dt.hour >= 18)))),
    ('거래주수', lambda x: x.dt.isocalendar().week.nunique()),
    ('거래기간일수', lambda x: (x.max() - x.min()).days + 1),

]


tmp = train_tr.groupby('ID')["구매일시"].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left',on="ID")
train_ft.head()

tmp = test_tr.groupby('ID')["구매일시"].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left',on="ID")
test_ft.head()

,ID,구매횟수,내점일수,구매주기,주말방문비율,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,...,월별_구매총액_12,월요일방문비율,화요일방문비율,수요일방문비율,목요일방문비율,금요일방문비율,토요일방문비율,일요일방문비율,거래주수,거래기간일수
0,test_0,7,5,32,0.571429,0.285714,0.285714,0.428571,0.000000,1,...,0,0.142857,0.285714,0.00000,0.00000,0.000000,0.285714,0.285714,5,164
1,test_1,4,3,54,0.250000,0.000000,0.000000,0.750000,0.250000,0,...,0,0.500000,0.000000,0.25000,0.00000,0.000000,0.250000,0.000000,3,165
2,test_2,4,2,8,0.000000,0.000000,0.600000,0.400000,0.000000,0,...,0,0.600000,0.000000,0.00000,0.40000,0.000000,0.000000,0.000000,2,17
3,test_3,28,16,16,0.031250,0.062500,0.468750,0.343750,0.125000,3,...,198000,0.062500,0.062500,0.03125,0.53125,0.281250,0.000000,0.031250,14,272
4,test_4,19,9,29,0.476190,0.380952,0.285714,0.238095,0.095238,6,...,0,0.047619,0.238095,0.00000,0.00000,0.238095,0.047619,0.428571,7,266


In [ ]:
train_tr['방문일자'] = train_tr['구매일시'].dt.date
test_tr['방문일자'] = test_tr['구매일시'].dt.date
train_tr.shape, test_tr.shape

((523105, 8), (441196, 8))

In [ ]:
하루_구매_시간_간격_train = train_tr.groupby(['ID', '방문일자'])['구매일시'].max() - train_tr.groupby(['ID', '방문일자'])['구매일시'].min()
하루_구매_시간_간격_train = 하루_구매_시간_간격_train.reset_index().rename(columns = {'구매일시' : '하루 구매 시간 간격'})

하루_구매_시간_간격_test = test_tr.groupby(['ID', '방문일자'])['구매일시'].max() - test_tr.groupby(['ID', '방문일자'])['구매일시'].min()
하루_구매_시간_간격_test = 하루_구매_시간_간격_test.reset_index().rename(columns = {'구매일시' : '하루 구매 시간 간격'})

하루_구매_시간_간격_train.shape, 하루_구매_시간_간격_test.shape

((271851, 3), (229253, 3))

In [ ]:
하루_구매_시간_간격_평균_train = 하루_구매_시간_간격_train.groupby("ID")["하루 구매 시간 간격"].mean()
하루_구매_시간_간격_평균_test = 하루_구매_시간_간격_test.groupby("ID")["하루 구매 시간 간격"].mean()

하루_구매_시간_간격_평균_train.shape, 하루_구매_시간_간격_평균_test.shape

((14940,), (12225,))

In [ ]:
train_ft = pd.merge(train_ft, 하루_구매_시간_간격_평균_train, on="ID", how = "left")
test_ft = pd.merge(test_ft, 하루_구매_시간_간격_평균_test, on="ID", how = "left")

train_ft.shape, test_ft.shape

((14940, 357), (12225, 357))

In [ ]:
agg_list = [
        ('최대구매액 - 최소구매액', lambda x : x.max() - x[x>0].min())
    ]
tmp = train_tr.groupby('ID')["구매가격"].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left')
train_ft.head()

tmp = test_tr.groupby('ID')["구매가격"].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left')
test_ft.head()

,ID,구매횟수,내점일수,구매주기,주말방문비율,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,...,화요일방문비율,수요일방문비율,목요일방문비율,금요일방문비율,토요일방문비율,일요일방문비율,거래주수,거래기간일수,하루 구매 시간 간격,최대구매액 - 최소구매액
0,test_0,7,5,32,0.571429,0.285714,0.285714,0.428571,0.000000,1,...,0.285714,0.00000,0.00000,0.000000,0.285714,0.285714,5,164,NaT,83357
1,test_1,4,3,54,0.250000,0.000000,0.000000,0.750000,0.250000,0,...,0.000000,0.25000,0.00000,0.000000,0.250000,0.000000,3,165,NaT,80400
2,test_2,4,2,8,0.000000,0.000000,0.600000,0.400000,0.000000,0,...,0.000000,0.00000,0.40000,0.000000,0.000000,0.000000,2,17,NaT,288000
3,test_3,28,16,16,0.031250,0.062500,0.468750,0.343750,0.125000,3,...,0.062500,0.03125,0.53125,0.281250,0.000000,0.031250,14,272,NaT,395960
4,test_4,19,9,29,0.476190,0.380952,0.285714,0.238095,0.095238,6,...,0.238095,0.00000,0.00000,0.238095,0.047619,0.428571,7,266,NaT,359800


0.7103883909249868(누구 거지)

In [ ]:
#월별 구매가격
train_tr["구매일시"] = pd.to_datetime(train_tr["구매일시"])
test_tr["구매일시"] = pd.to_datetime(test_tr["구매일시"])
# 월을 추출
train_tr['구매월'] = train_tr['구매일시'].dt.month  # 월 단위로 추출

# train_tmp 피벗 테이블 생성
train_tmp = pd.pivot_table(train_tr,index="ID",columns="구매월",values="구매가격",aggfunc="sum",fill_value=0).add_prefix("월별_구매총액_")

# # 컬럼 이름을 명확하게 변경
# train_tmp.columns.name = None  # 컬럼 이름 제거
# train_tmp.reset_index(inplace=True)  # 인덱스 리셋

# train_ft에 병합
train_ft = train_ft.merge(train_tmp, how="left", on="ID")
train_ft.head()

,ID,구매횟수,내점일수,구매주기,주말방문비율,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,...,월별_구매총액_3,월별_구매총액_4,월별_구매총액_5,월별_구매총액_6,월별_구매총액_7,월별_구매총액_8,월별_구매총액_9,월별_구매총액_10,월별_구매총액_11,월별_구매총액_12
0,train_0,16,11,23,0.250000,0.050000,0.250000,0.400000,0.300000,3,...,0,0,43500,1750000,334000,0,525300,0,1281000,-72600
1,train_1,41,21,16,0.023810,0.357143,0.166667,0.357143,0.119048,3,...,425000,422000,850400,72000,101500,530525,843450,67000,230150,1173000
2,train_2,102,56,6,0.210526,0.464912,0.140351,0.175439,0.219298,0,...,1150456,2895168,1017000,3338109,456000,354000,4577000,1689000,997000,184500
3,train_3,191,92,3,0.189573,0.379147,0.180095,0.236967,0.203791,3,...,5034820,8819200,3349279,1391000,2815863,179000,3044800,869000,1649100,3643800
4,train_4,55,27,11,0.258065,0.112903,0.612903,0.209677,0.064516,4,...,50816,0,1257606,-664000,1277956,727056,934748,217200,20063,0


In [ ]:
test_tr['구매월'] = test_tr['구매일시'].dt.month

# test_tmp 피벗 테이블 생성
test_tmp = pd.pivot_table(test_tr,index="ID",columns="구매월",values="구매가격",aggfunc="sum",fill_value=0).add_prefix("월별_구매총액_")

# # 컬럼 이름을 명확하게 변경
# test_tmp.columns.name = None  # 컬럼 이름 제거
# test_tmp.reset_index(inplace=True)  # 인덱스 리셋

# train_tmp와 test_tmp의 컬럼 정렬 및 결측치 처리
for col in train_tmp.columns:
    if col not in test_tmp.columns:
        test_tmp[col] = 0  # test_tmp에 없는 컬럼 추가

# train_tmp 컬럼 순서에 맞게 test_tmp 정렬
test_tmp = test_tmp[train_tmp.columns]

# test_ft에 병합
test_ft = test_ft.merge(test_tmp, how="left", on="ID")

train_ft.shape, test_ft.shape

((14940, 369), (12225, 369))

In [ ]:
# 월별 차이
for i in range(2, 13):
    column_name = f'월별_구매총액_변화량_{i}_{i-1}'
    train_ft[column_name] = train_ft[f'월별_구매총액_{i}'] - train_ft[f'월별_구매총액_{i-1}']
    test_ft[column_name] = test_ft[f'월별_구매총액_{i}'] - test_ft[f'월별_구매총액_{i-1}']
train_ft['월별_구매총액_변화량_1_12'] = train_ft['월별_구매총액_1'] - train_ft['월별_구매총액_12']
test_ft['월별_구매총액_변화량_1_12'] = test_ft['월별_구매총액_1'] - test_ft['월별_구매총액_12']

train_ft.shape, test_ft.shape

((14940, 381), (12225, 381))

In [ ]:
#전달과의 구매비율 차이
for i in range(2, 13):
    column_name = f'월별_구매총액_변화_비율{i}_{i-1}'
    train_ft[column_name] = train_ft[f'월별_구매총액_{i}'] / train_ft[f'월별_구매총액_{i-1}']
    test_ft[column_name] = test_ft[f'월별_구매총액_{i}'] / test_ft[f'월별_구매총액_{i-1}']
train_ft['월별_구매총액_변화량_1_12'] = train_ft['월별_구매총액_1'] / train_ft['월별_구매총액_12']
test_ft['월별_구매총액_변화량_1_12'] = test_ft['월별_구매총액_1'] / test_ft['월별_구매총액_12']

train_ft.shape, test_ft.shape

((14940, 392), (12225, 392))

In [ ]:
# train_tr['브랜드코드'].nunique(), test_tr['브랜드코드'].nunique()

(1830, 1800)

In [ ]:
# 추릴까?..?

In [ ]:
# 브랜드코드 = train_tr.groupby('브랜드코드')['ID'].count().sort_values(ascending=False).reset_index()
# 브랜드코드.columns = ['브랜드코드', '브랜드코드_구매횟수']
# 브랜드코드 = 브랜드코드[브랜드코드['브랜드코드_구매횟수'] > 500]

In [ ]:
# 브랜드_평균구매액 = train_tr.groupby(['ID', '브랜드코드'])['구매가격'].mean().sort_values(ascending=False).reset_index()
# 브랜드_평균구매액.columns = ['ID','브랜드코드', '브랜드코드_평균구매액']

In [ ]:
# 브랜드코드 = pd.merge(브랜드코드, 브랜드_평균구매액, on='브랜드코드', how='left')
# 브랜드코드

,브랜드코드,브랜드코드_구매횟수,브랜드코드_평균구매액
0,5100,119088,42186.780498
1,5127,8860,42206.512415
2,5173,6517,75900.494092
3,5116,5682,100313.269975
4,5138,5277,113373.791927
...,...,...,...
210,5489,509,47239.685658
211,5995,508,85736.023622
212,5290,504,270962.301587
213,5667,504,69820.277778


In [ ]:
#브랜드별 평균구매액
브랜드_평균구매액 = train_tr.groupby(['ID', '브랜드코드'])['구매가격'].mean().sort_values(ascending=False).reset_index()
브랜드_평균구매액.columns = ['ID','브랜드코드', '브랜드코드_평균구매액']
브랜드_평균구매액 = 브랜드_평균구매액.groupby('ID')['브랜드코드_평균구매액'].mean().reset_index()
train_ft = train_ft.merge(브랜드_평균구매액, on='ID', how='left')

브랜드_평균구매액 = test_tr.groupby(['ID', '브랜드코드'])['구매가격'].mean().sort_values(ascending=False).reset_index()
브랜드_평균구매액.columns = ['ID','브랜드코드', '브랜드코드_평균구매액']
브랜드_평균구매액 = 브랜드_평균구매액.groupby('ID')['브랜드코드_평균구매액'].mean().reset_index()
test_ft = test_ft.merge(브랜드_평균구매액, on='ID', how='left')

train_ft.shape, test_ft.shape

((14940, 393), (12225, 393))

# New

#지점코드별 확인

In [ ]:
#중분류 다시 분류

In [ ]:
train_tr['중분류'].sort_values().unique()

array(['DC캐주얼', 'GBR  지원', 'L_B침구', 'NB제화', 'NB핸드백', 'N_B침구', 'TOP디자이너',
       'TV.VTR', 'TV_VCR', 'TV_VTR', '가방', '가스기기', '가스렌지', '가전특정', '가정잡화',
       '건강식품', '건강용품', '건식품', '건어물', '곡물', '골프(LC)', '골프(NB)', '골프(국내)',
       '골프(단품)', '골프(수입)', '골프(용품)', '골프단품', '골프용품', '골프웨어', '과자', '과자류',
       '교복행사', '구두수선', '구두임대', '구두행사', '국내부띠끄', '국내종합화장품', '국내화장품',
       '국산화장품', '그라스', '기타식품', '남성구두', '남성잡화', '내셔날', '내셔널', '내의', '냉난방',
       '냉동식품', '냉장고.세탁기', '냉장고_세탁기', '냉장식품', '넥타이', '뉴베이직캐주얼', '니트',
       '니트웨어', '단품', '단품(트래디셔널)', '단품_행사', '단품_행사(캐릭터)', '단품_행사(트래디셔널)',
       '단품_행사1', '단품_행사2', '단품행사', '드레스구두', '디자이너', '디자이너니트', '디자이너부띠끄',
       '디자이너숍', '디자이너캐릭터', '디자이너캐주얼', '라디오.카세트', '라이센스', '란제리', '레이디숍A',
       '레코드(문화)', '레포츠', '레포츠단품', '로얄', '로얄부틱2F', '로얄수입행사', '마담SIZE',
       '마춤', '머플러', '면류', '명품', '모자', '모피', '모피.피혁', '모피_피혁', '모피니트',
       '문구', '문구(문화)', '문구_팬시', '문화', '미시케쥬얼', '미씨캐릭터', '베이직캐주얼', '보석',
       '부띠끄', '부띠끄행사', '브랜드침구', '비디오', '사무용품', '상품개발지원', '상품군미지

In [ ]:
train_tr['대분류'].nunique()

28

In [ ]:
train_tr['지점코드'].unique()

array(['A144000', 'A112000', 'A373000', 'A202000'], dtype=object)

In [ ]:
train_tr[train_tr['중분류'].str.contains('소품')]['지점코드'].unique()

array(['A112000', 'A202000', 'A144000'], dtype=object)

In [ ]:
train_tr[train_tr['지점코드'] == 'A373000']['중분류'].nunique(), train_tr[train_tr['지점코드'] == 'A373000']['중분류'].unique()

(119,
 array(['아동복', '상품군미지정', '세탁기_냉장고', '완구(문화)', '야채', '면류', '패션시계', '골프(국내)',
        '진캐쥬얼', '색조화장품', '주방용품', '아웃도어', '핸드백', '과자류', '란제리', '곡물', '니트웨어',
        '수입향수', '명품', '단품_행사(트래디셔널)', '신생아', '팬시코너(문화)', '수입종합화장품', '타운웨어',
        '영캐주얼', '트래디셔널', '캐릭터', '단품', '로얄부틱2F', '수영복', '생선', '용기보증', '부띠끄',
        '모자', '캐릭터캐주얼', '슈즈', '수입액세서리', '여성구두', '골프(수입)', '잡화(문화)', '수입',
        '라이센스', '구두임대', '건강식품', '즉석조리', '청과', '셔츠', '일용잡화', '냉장식품',
        '레코드(문화)', '가전특정', '우산_장갑', '일반악세사리', '카페트', '욕실용품', '남성구두', '가방',
        '마춤', '브랜드침구', '보석', '건식품', '헤어액세사리', '전문가구(가구)', '가정잡화', '임대화장품',
        '준보석', '음료', '스타킹', '차류', '디자이너캐주얼', '내셔날', '양말', '임대골프',
        '카세트_전화기', '크리스탈', '일반조리', 'TV_VTR', '소형전기', '정육', '인스턴트식품', '손수건',
        '식기', '스포츠웨어', '스포츠용품', '기타식품', '화장잡화', '소형가구(가구)', '문구(문화)',
        '모피니트', '수입도자기', '행사', '수입행사', '수예침장', '특정', '스카프', '핸드백임대', '주류',
        '조미료', '냉동식품', '냉난방', '교복행사', '그라스', '장신구행사업체', '통병조림', '영트랜디',
        '영창(문화)', '부띠끄행사', '일반식품명품', '정

In [ ]:
train_tr[train_tr['지점코드'] == 'A112000']['중분류'].nunique(), train_tr[train_tr['지점코드'] == 'A112000']['중분류'].unique()

(132,
 array(['용기보증', '전화기_카세트', '일용잡화', '아동', '수영복', '야채', '수입종합화장품', 'TV_VCR',
        '상품군미지정', '차류', '주방용품', '정육', '양말', '니트웨어', '문화', '우산장갑',
        '칼라드래디셔널', '스카프', '남성잡화', '소형취사가전', '영캐주얼', '완구', '진캐주얼', '청과',
        '디자이너', '디자이너부띠끄', '셔츠', '패션란제리', '손수건', '스포츠웨어', '수입ACC', '욕실용품',
        'NB제화', 'L_B침구', '란제리', '트레디셔널캐주얼', '싸롱화', '핸드백', '화장품', '미씨캐릭터',
        '냉장식품', '레포츠단품', '인텔리젼스캐주얼', '골프웨어', '스포츠캐주얼', '건강식품', '크리스탈',
        '베이직캐주얼', '토탈', '스포츠슈즈', '국내종합화장품', '유아', '피혁토탈(B2)', '단품_행사1',
        '영커리어캐주얼', '초도자기', '단품_행사2', '음료', '보석', '생선', '수예_인테리어소품',
        '스포츠용품', '단품', '트렌드캐주얼', '어덜트', '캐릭터', '가정잡화', '향수', '패스트푸드',
        '트.단품', '일반조리', '유니섹스캐주얼', '홈데코', '시계', '골프용품', '단품_행사', '과자류',
        '장신구', '핸드백행사', '가방', '패션ACC', '주류', '원목_주니어', '아웃도어', '헤어ACC',
        '식탁_소품', '냉장고_세탁기', '즉석조리', 'N_B침구', '뉴베이직캐주얼', '색조화장품', '스타킹',
        '수입명품', '수입도자기', '컴퓨터', '수입의류행사', '건식품', '준보석', '모자', '라이센스', '면류',
        '구두행사', '인스탄트식품', '침대', '곡물', '종합_수입', '스포츠단품', '기타

In [ ]:
train_tr[train_tr['지점코드'] == 'A202000']['중분류'].nunique(), train_tr[train_tr['지점코드'] == 'A202000']['중분류'].unique()

(136,
 array(['아동복', '단품', '캐릭터캐쥬얼', '문구_팬시', '유아용품', '골프(LC)', '행사_단품', '용기보증',
        '완구', '향수', '수입종합화장품', '란제리', '셔츠', '영캐쥬얼', '행사', '생선', '아동잡화',
        '캐리어캐쥬얼', '진캐쥬얼', '침구', '일반조리', '차류', '캐릭터슈즈', '패션시계', '냉장식품',
        '캐릭터', '스타킹', '정육', '즉석조리', '야채', '넥타이', '국산화장품', '청과', '색조화장품',
        '트래디셔널캐쥬얼', '타운', '잡화토탈', '엘레강스부틱', '일용잡화', '주방용품', '니트', '행사핸드백',
        '어덜트', '우산_장갑', '기타식품', '라이센스', '과자', '유아복', '선글래스', '행사소품', '조미료',
        '스포츠단품', '아동단품', '임대핸드백', '핸드백', '패션악세사리', '헤어악세사리', '취사소형',
        '초도자기', '욕실용품', '스포츠웨어', '엘레강스', '수입슈즈', '아웃도어', '타운웨어', '토탈',
        '수영복', '내셔널', '원목(주니어)', '스카프', '양말', '골프(단품)', '수입악세사리', '의류기타',
        '상품군미지정', '비디오', '임대슈즈', '건강식품', '건어물', '손수건', '골프(NB)', '스포츠슈즈',
        '주류', '보석', '스포츠용퓸', '디자이너캐릭터', '트래디셔널', '패션잡화', '토탈부틱', '행사슈즈',
        '가정잡화', '수예', '크리스탈', '디자이너', '레포츠', '모자', '침대', '스키', '면류', '곡물',
        '냉동식품', '인스탄트식품', '전화기_카세트', '준보석', '카메라', '수입', '음료', '해외SHOP',
        '통병조림', '훼미닌부틱', '냉장고_세탁기', '건강용품',

In [ ]:
train_tr[train_tr['지점코드'] == 'A144000']['중분류'].nunique(), train_tr[train_tr['지점코드'] == 'A144000']['중분류'].unique()

(128,
 array(['차류', '화장잡화', '수입종합화장품', '테이프', '진케주얼', '용기보증', '트래디셔널', '야채',
        '수입부띠끄', '골프웨어', '영트랜드', '곡물', '스타킹', '레이디숍A', '하이캐쥬얼', '유아복',
        '헤어ACC', '타운단품', '란제리', '청과', '냉장식품', '아동', '잡화', '캐쥬얼구두', '내의',
        '일반조리', '스카프', '상품군미지정', '디자이너숍', '썬그라스', '캐릭터캐주얼', '완구', '수입의류',
        '준보석', '욕실용품', '캐릭터캐쥬얼', '수입캐주얼', '즉석조리', '일용잡화', '국내화장품', '임대구두',
        '미시케쥬얼', '취사소형', '스포츠웨어', '넥타이', '모자', '피혁B행사', '수입구두', '팬시',
        '색조화장품', '영캐쥬얼', '디자이너니트', '건식품', '핸드백', '크리스탈', '건강식품', '양말',
        '우산_장갑', '페레  지원', '국내부띠끄', '초도자기', '음료', '라이센스', '스포츠슈즈', '드레스구두',
        '셔츠', '상품개발지원', '문구', '패션시계', '주방용품', '냉장고.세탁기', '내셔날', '과자류',
        '면류', '수영복', '가정잡화', '인텔리젼스', '수입향수', '수입피혁', '정육', '주류', '보석',
        '일반ACC', '수입ACC', '피혁A행사', '골프단품', '생선', '아동특선', '임대골프', '조미료',
        '가방', '손수건', '수입도자기', '전화기', '냉난방', '모피', '일반식품명품', '수예행사', '수입',
        '머플러', '침구', '로얄수입행사', '통병조림', '라디오.카세트', '캐릭터', '냉동식품', 'DC캐주얼',
        '용품', '기타식품', '마춤', 'GBR  지원', '인스탄트식품', '

#중분류별 지점코드 개수

In [ ]:
개수 = train_tr.groupby('중분류')['지점코드'].nunique().reset_index()
개수

,중분류,지점코드
0,DC캐주얼,1
1,GBR 지원,1
2,L_B침구,1
3,NB제화,1
4,NB핸드백,1
...,...,...
298,헤어액세사리,1
299,홈데코,1
300,화장잡화,2
301,화장품,1


In [ ]:
개수[개수['지점코드'] == 1].shape, 개수[개수['지점코드'] == 2].shape, 개수[개수['지점코드'] == 3].shape, 개수[개수['지점코드'] == 4].shape

((202, 2), (36, 2), (19, 2), (46, 2))

In [ ]:
개수[개수['지점코드'] == 4]['중분류'].unique()

array(['가방', '가정잡화', '건강식품', '곡물', '기타식품', '냉난방', '냉동식품', '냉장식품', '라이센스',
       '란제리', '머플러', '면류', '모자', '보석', '상품군미지정', '색조화장품', '생선', '셔츠',
       '손수건', '수영복', '수입도자기', '수입종합화장품', '스카프', '스키', '스타킹', '스포츠웨어',
       '야채', '양말', '욕실용품', '용기보증', '음료', '일반조리', '일용잡화', '정육', '조미료',
       '주류', '주방용품', '준보석', '즉석조리', '차류', '청과', '캐릭터', '크리스탈', '통병조림',
       '특정', '핸드백'], dtype=object)

In [ ]:
개수[개수['지점코드'] == 3]['중분류'].unique()

array(['건식품', '과자류', '내셔날', '넥타이', '단품', '디자이너', '수입', '스포츠단품', '스포츠슈즈',
       '아웃도어', '오디오', '완구', '우산_장갑', '인스탄트식품', '초도자기', '카페트', '트래디셔널',
       '패션시계', '행사'], dtype=object)

In [ ]:
개수[개수['지점코드'] == 2]['중분류'].unique()

array(['TV_VTR', '가스기기', '골프웨어', '냉장고_세탁기', '니트웨어', '디자이너캐릭터', '마춤',
       '사무용품', '수입ACC', '수입명품', '수입향수', '스포츠용품', '아동', '아동복', '어덜트',
       '영캐주얼', '영캐쥬얼', '유아복', '일반식품명품', '임대골프', '임대핸드백', '잡화', '전화기_카세트',
       '정장행사', '진캐쥬얼', '취사소형', '침구', '침대', '캐릭터캐주얼', '캐릭터캐쥬얼', '타운웨어',
       '토탈', '피혁', '향수', '헤어ACC', '화장잡화'], dtype=object)

In [ ]:
개수[개수['지점코드'] == 1]['중분류'].unique()

array(['DC캐주얼', 'GBR  지원', 'L_B침구', 'NB제화', 'NB핸드백', 'N_B침구', 'TOP디자이너',
       'TV.VTR', 'TV_VCR', '가스렌지', '가전특정', '건강용품', '건어물', '골프(LC)',
       '골프(NB)', '골프(국내)', '골프(단품)', '골프(수입)', '골프(용품)', '골프단품', '골프용품',
       '과자', '교복행사', '구두수선', '구두임대', '구두행사', '국내부띠끄', '국내종합화장품', '국내화장품',
       '국산화장품', '그라스', '남성구두', '남성잡화', '내셔널', '내의', '냉장고.세탁기', '뉴베이직캐주얼',
       '니트', '단품(트래디셔널)', '단품_행사', '단품_행사(캐릭터)', '단품_행사(트래디셔널)', '단품_행사1',
       '단품_행사2', '단품행사', '드레스구두', '디자이너니트', '디자이너부띠끄', '디자이너숍', '디자이너캐주얼',
       '라디오.카세트', '레이디숍A', '레코드(문화)', '레포츠', '레포츠단품', '로얄', '로얄부틱2F',
       '로얄수입행사', '마담SIZE', '명품', '모피', '모피.피혁', '모피_피혁', '모피니트', '문구',
       '문구(문화)', '문구_팬시', '문화', '미시케쥬얼', '미씨캐릭터', '베이직캐주얼', '부띠끄',
       '부띠끄행사', '브랜드침구', '비디오', '상품개발지원', '선글라스', '선글래스', '세탁기_냉장고', '소파',
       '소형가구(가구)', '소형전기', '소형취사가전', '쇼파', '수예', '수예_인테리어소품', '수예침장',
       '수예행사', '수입_종합가구', '수입구두', '수입부띠끄', '수입슈즈', '수입악세사리', '수입액세서리',
       '수입의류', '수입의류행사', '수입캐주얼', '수입피혁', '수입행사', '슈즈', '스포츠용퓸',

# 확인코드

In [ ]:
train_tr[train_tr['중분류'].str.contains('트래디셔널')]['지점코드'].unique()

array(['A144000', 'A373000', 'A202000'], dtype=object)

In [ ]:
train_tr[train_tr['중분류'].str.contains('트레디셔널')]['지점코드'].unique()

array(['A112000'], dtype=object)

In [ ]:
train_tr[train_tr['중분류']==('문구')]['지점코드'].unique()

array(['A144000'], dtype=object)

In [ ]:
train_tr[train_tr['중분류']==('문구(문화)')]['지점코드'].unique()

array(['A373000'], dtype=object)

In [ ]:
train_tr[train_tr['중분류']==('문구_팬시')]#['지점코드'].unique()

,ID,구매일시,지점코드,대분류,중분류,브랜드코드,구매가격,방문일자,구매월
36,train_2935,2004-05-01 11:00:00,A202000,가정용품,문구_팬시,5156,8800,2004-05-01,5
173,train_12701,2004-05-01 12:10:00,A202000,가정용품,문구_팬시,5320,8100,2004-05-01,5
454,train_462,2004-05-01 14:10:00,A202000,가정용품,문구_팬시,5532,33000,2004-05-01,5
835,train_7035,2004-05-01 16:20:00,A202000,가정용품,문구_팬시,5668,18500,2004-05-01,5
1073,train_2117,2004-05-01 17:30:00,A202000,아동,문구_팬시,5468,16900,2004-05-01,5
...,...,...,...,...,...,...,...,...,...
519560,train_10793,2005-04-27 15:40:00,A202000,가정용품,문구_팬시,6144,14000,2005-04-27,4
521767,train_10550,2005-04-29 12:20:00,A202000,가정용품,문구_팬시,5368,24000,2005-04-29,4
522056,train_270,2005-04-29 14:20:00,A202000,가정용품,문구_팬시,5368,3000,2005-04-29,4
522356,train_4165,2005-04-29 15:50:00,A202000,가정용품,문구_팬시,5320,13900,2005-04-29,4


In [ ]:
train_tr[train_tr['중분류']==('문화')]#['지점코드'].unique()

,ID,구매일시,지점코드,대분류,중분류,브랜드코드,구매가격,방문일자,구매월
88,train_14467,2004-05-01 11:30:00,A112000,아동문화,문화,5225,15500,2004-05-01,5
113,train_12575,2004-05-01 11:40:00,A112000,아동문화,문화,5252,69000,2004-05-01,5
779,train_13775,2004-05-01 16:00:00,A112000,아동문화,문화,5517,8000,2004-05-01,5
869,train_2226,2004-05-01 16:30:00,A112000,아동문화,문화,5225,5800,2004-05-01,5
888,train_12575,2004-05-01 16:40:00,A112000,아동문화,문화,5252,-69000,2004-05-01,5
...,...,...,...,...,...,...,...,...,...
522343,train_12597,2005-04-29 15:43:00,A112000,아동문화,문화,5607,26000,2005-04-29,4
522470,train_8343,2005-04-29 16:13:00,A112000,아동문화,문화,5647,18000,2005-04-29,4
522527,train_13419,2005-04-29 16:30:00,A112000,아동문화,문화,5272,10000,2005-04-29,4
522790,train_11204,2005-04-29 17:43:00,A112000,아동문화,문화,5225,14720,2005-04-29,4


In [ ]:
train_tr[train_tr['중분류']==('원목(주니어)')]

,ID,구매일시,지점코드,대분류,중분류,브랜드코드,구매가격,방문일자,구매월
1422,train_10145,2004-05-02 10:44:00,A202000,가정용품,원목(주니어),5815,1323000,2004-05-02,5
3662,train_242,2004-05-03 16:40:00,A202000,가정용품,원목(주니어),5183,54400,2004-05-03,5
9007,train_9956,2004-05-06 17:30:00,A202000,가정용품,원목(주니어),6196,178200,2004-05-06,5
21986,train_6901,2004-05-14 17:02:00,A202000,가정용품,원목(주니어),6315,1310000,2004-05-14,5
23180,train_6901,2004-05-16 13:34:00,A202000,가정용품,원목(주니어),6315,-1310000,2004-05-16,5
...,...,...,...,...,...,...,...,...,...
501070,train_10242,2005-04-14 17:40:00,A202000,가정용품,원목(주니어),6399,416000,2005-04-14,4
502178,train_7333,2005-04-15 11:20:00,A202000,가정용품,원목(주니어),6196,357200,2005-04-15,4
503517,train_14465,2005-04-15 15:00:00,A202000,가정용품,원목(주니어),6399,136500,2005-04-15,4
503518,train_14465,2005-04-15 15:00:00,A202000,가정용품,원목(주니어),6399,-136500,2005-04-15,4


In [ ]:
x = train_tr[train_tr['지점코드']==('A112000')]
x[x['중분류'].str.contains('니트')]['중분류'].unique()

array(['니트웨어'], dtype=object)

In [ ]:
train_tr[train_tr['대분류']==('아동_스포츠')]['중분류'].unique()

array(['아동복', '완구(문화)', '아웃도어', '신생아', '팬시코너(문화)', '단품', '수영복', '슈즈',
       '잡화(문화)', '레코드(문화)', '스포츠웨어', '스포츠용품', '문구(문화)', '상품군미지정', '교복행사',
       '용기보증', '영창(문화)', '스키'], dtype=object)

In [ ]:
train_tr[train_tr['대분류']==('스포츠캐쥬얼')]['중분류'].unique()

array(['수영복', '골프(LC)', '스포츠웨어', '트레디셔널캐주얼', '트래디셔널캐쥬얼', '골프웨어', '스포츠슈즈',
       '스포츠용품', '스포츠단품', '아웃도어', '골프용품', '골프(단품)', '단품', '골프(NB)',
       '스포츠용퓸', '용기보증', '스키', '수입', '캐쥬얼단품', '골프(용품)', '캐주얼단품', '진캐쥬얼'],
      dtype=object)

In [ ]:
train_tr[train_tr['중분류']==('용기보증')]['대분류'].unique()

array(['공산품', '공산품파트', '생식품', '생식품파트', '케주얼_구두_아동', '영캐릭터', '스포츠캐쥬얼',
       '가정용품파트', '영라이브', '가정용품', '여성의류파트', '명품잡화', '영플라자', '남성정장스포츠',
       '여성캐주얼', '여성정장', '아동문화', '골프_유니캐쥬얼', '잡화', '아동_스포츠', '아동', '잡화파트',
       '여성캐쥬얼', '남성의류', '영어덜트캐쥬얼', '패션잡화', '로얄부틱', '로얄부띠끄'], dtype=object)

In [ ]:
#모델링에서 pivot 추가로 생성

In [ ]:
# #첫구매와 최근구매 차이 -> 결측치 너무 심함
# train_ft['최근 구매월와 첫 구매월 차이']

In [ ]:
train_ft

,ID,구매횟수,내점일수,구매주기,주말방문비율,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,...,월별_구매총액_3,월별_구매총액_4,월별_구매총액_5,월별_구매총액_6,월별_구매총액_7,월별_구매총액_8,월별_구매총액_9,월별_구매총액_10,월별_구매총액_11,월별_구매총액_12
0,train_0,16,11,23,0.250000,0.050000,0.250000,0.400000,0.300000,3,...,0,0,43500,1750000,334000,0,525300,0,1281000,-72600
1,train_1,41,21,16,0.023810,0.357143,0.166667,0.357143,0.119048,3,...,425000,422000,850400,72000,101500,530525,843450,67000,230150,1173000
2,train_2,102,56,6,0.210526,0.464912,0.140351,0.175439,0.219298,0,...,1150456,2895168,1017000,3338109,456000,354000,4577000,1689000,997000,184500
3,train_3,191,92,3,0.189573,0.379147,0.180095,0.236967,0.203791,3,...,5034820,8819200,3349279,1391000,2815863,179000,3044800,869000,1649100,3643800
4,train_4,55,27,11,0.258065,0.112903,0.612903,0.209677,0.064516,4,...,50816,0,1257606,-664000,1277956,727056,934748,217200,20063,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14935,train_14935,16,10,24,0.210526,0.000000,0.578947,0.052632,0.368421,2,...,0,0,0,46000,81200,462000,120000,0,0,366400
14936,train_14936,8,8,29,0.125000,0.125000,0.625000,0.250000,0.000000,0,...,54000,0,0,0,220000,0,1894000,0,52000,0
14937,train_14937,22,12,26,0.083333,0.208333,0.458333,0.208333,0.125000,1,...,58000,150870,0,141810,71470,286528,0,110978,24742,53500
14938,train_14938,31,12,28,0.151515,0.393939,0.393939,0.212121,0.000000,3,...,58000,726100,920000,198000,851500,0,788000,0,595000,0


In [ ]:
#결혼관련 제품 구매 - 가격 변동성

# 이름 바꾸기

In [ ]:
train_tr.shape

(523105, 7)

In [ ]:
train_tr["수정_중분류"] = train_tr["중분류"]
test_tr["수정_중분류"] = test_tr["중분류"]
train_tr.shape, test_tr.shape

((523105, 8), (441196, 8))

In [ ]:
names_list = {
    #1개 위쪽
    "국내화장품" : ['국내종합화장품', '국내화장품','국산화장품'],
    "단품행사" : ['단품_행사', '단품_행사1','단품_행사2', '단품행사',  '행사_단품'],
    "레포츠" : ['레포츠', '레포츠단품'],
    "모피" : ['모피', '모피.피혁', '모피_피혁', '모피니트'],
    "문구" : ['문구', '문구(문화)', '문구_팬시'],
    "선글라스" : ['선글라스', '선글래스', '썬그라스'],
    "수입구두" : ['수입구두', '수입슈즈'],
    #2개
    'TV': ['TV_VTR', 'TV.VTR', 'TV_VCR'],
    '냉장고세탁기': ['냉장고세탁기', '냉장고.세탁기', '세탁기냉장고'],
    '니트': ['니트웨어', '니트'],
    '악세서리': ['수입ACC', '수입악세사리', '수입액세서리'],
    '아동': ['아동', '유아'],
    '아동복': ['아동복', '유아복'],
    '영캐주얼': ['영캐주얼', '영캐쥬얼'],
    '임대핸드백': ['임대핸드백', '핸드백임대'],
    '전화기카세트': ['전화기카세트', '카세트_전화기'],
    '진캐주얼': ['진캐쥬얼', '진캐주얼', '진케주얼'],
    '소형취사가전': ['취사소형', '소형취사가전'],
    '침구': ['침구', '직수입침구'],
    '캐릭터캐주얼': ['캐릭터캐주얼', '캐릭터캐쥬얼'],
    '타운웨어': ['타운웨어', '타운', '타운단품'],
    '헤어악세서리': ['헤어ACC', '헤어악세사리', '헤어액세사리'],
    #3개
    '건식품':['건어물'],
    '과자류':['과자'],
    '내셔날':['내셔널'],
    '오디오':[' 라디오.카세트'],
    '완구':['완구(문화)'],
    '우산장갑':['우산장갑'],
    '인스탄트식품':['인스턴트식품'],
    '카페트':['카페트대자리'],
    '트래디셔널':['트래디셔널캐쥬얼', '트레디셔널캐주얼'],
    #1개 아래쪽
    '소형가구': ['소형가구(가구)', '소형전기', '소형취사가전'],
    '소파': ['쇼파', '소파'],
    '캐주얼단품':['캐주얼단품', '캐쥬얼단품'],
    '영트랜드': ['영트랜드', '영트랜디'],
    '수입악세서리' : ['수입악세사리', '수입액세서리'],
    '원목(주니어)': ['원목(주니어)', '원목_주니어'],
    '타운': ['타운', '타운단품'],
    '일반악세서리': ['일반ACC', '일반악세사리'],
    '임대슈즈': ['임대구두', '임대슈즈']
}
for best, names in names_list.items():
    filtered_rows = train_tr["수정_중분류"].isin(names)
    train_tr.loc[filtered_rows, "수정_중분류"] = best



In [ ]:
for best, names in names_list.items():
    filtered_rows = test_tr["수정_중분류"].isin(names)
    test_tr.loc[filtered_rows, "수정_중분류"] = best

In [ ]:
train_tr['수정_중분류'].nunique()

255

In [ ]:
test_tr['수정_중분류'].nunique()

254

In [ ]:
agg_list = [
            ('수정_중분류_nunique', 'nunique'),
            ('주구매_수정_중분류', lambda x: x.mode()[0]),
            ]
tmp = train_tr.groupby('ID')["수정_중분류"].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left')
train_ft.head()

,ID,구매횟수,내점일수,구매주기,주말방문비율,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,...,pivot_브랜드코드수_지점A112000,pivot_브랜드코드수_지점A144000,pivot_브랜드코드수_지점A202000,pivot_브랜드코드수_지점A373000,pivot_금액합_지점A112000,pivot_금액합_지점A144000,pivot_금액합_지점A202000,pivot_금액합_지점A373000,수정_중분류_nunique,주구매_수정_중분류
0,train_0,16,11,23,0.250000,0.050000,0.250000,0.400000,0.300000,3,...,8,5,0,0,1242800,2958400,0,0,11,골프웨어
1,train_1,41,21,16,0.023810,0.357143,0.166667,0.357143,0.119048,3,...,22,2,0,8,4172100,50400,0,820525,23,영캐주얼
2,train_2,102,56,6,0.210526,0.464912,0.140351,0.175439,0.219298,0,...,0,9,0,51,0,2070000,0,20065386,35,아웃도어
3,train_3,191,92,3,0.189573,0.379147,0.180095,0.236967,0.203791,3,...,16,83,0,12,2222800,28663962,0,4708000,57,수입종합화장품
4,train_4,55,27,11,0.258065,0.112903,0.612903,0.209677,0.064516,4,...,0,18,0,6,0,2799803,0,1189063,24,로얄부틱2F


In [ ]:
tmp = test_tr.groupby('ID')["수정_중분류"].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left')
test_ft.head()

,ID,구매횟수,내점일수,구매주기,주말방문비율,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,...,pivot_브랜드코드수_지점A112000,pivot_브랜드코드수_지점A144000,pivot_브랜드코드수_지점A202000,pivot_브랜드코드수_지점A373000,pivot_금액합_지점A112000,pivot_금액합_지점A144000,pivot_금액합_지점A202000,pivot_금액합_지점A373000,수정_중분류_nunique,주구매_수정_중분류
0,test_0,7,5,32,0.571429,0.285714,0.285714,0.428571,0.000000,1,...,0,0,5,0,0,0,429579,0,4,용기보증
1,test_1,4,3,54,0.250000,0.000000,0.000000,0.750000,0.250000,0,...,1,0,0,3,62000,0,0,228600,4,남성구두
2,test_2,4,2,8,0.000000,0.000000,0.600000,0.400000,0.000000,0,...,0,5,0,0,0,607000,0,0,4,수입종합화장품
3,test_3,28,16,16,0.031250,0.062500,0.468750,0.343750,0.125000,3,...,19,0,0,0,2305740,0,0,0,12,수입종합화장품
4,test_4,19,9,29,0.476190,0.380952,0.285714,0.238095,0.095238,6,...,6,4,0,4,833100,157700,0,577890,10,칼라드래디셔널


In [ ]:
train_tmp = pd.pivot_table(train_tr,index="ID",columns="수정_중분류",values="구매가격",aggfunc="count",fill_value=0).add_prefix("수정_중_pivot_cnt_").reset_index()
train_ft = train_ft.merge(train_tmp,how="left",on="ID")
train_ft.head()
test_tmp = pd.pivot_table(test_tr,index="ID",columns="수정_중분류",values="구매가격",aggfunc="count",fill_value=0).add_prefix("수정_중_pivot_cnt_").reset_index()
test_tmp.head()

for col in train_tmp.columns:
    if col not in test_tmp.columns:
        test_tmp[col] = 0

test_tmp = test_tmp[train_tmp.columns]

test_ft = test_ft.merge(test_tmp,how="left",on="ID")
test_ft.head()

,ID,구매횟수,내점일수,구매주기,주말방문비율,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,...,수정_중_pivot_cnt_행사구두(5F),수정_중_pivot_cnt_행사소품,수정_중_pivot_cnt_행사슈즈,수정_중_pivot_cnt_행사핸드백,수정_중_pivot_cnt_향수,수정_중_pivot_cnt_헤어악세서리,수정_중_pivot_cnt_홈데코,수정_중_pivot_cnt_화장잡화,수정_중_pivot_cnt_화장품,수정_중_pivot_cnt_훼미닌부틱
0,test_0,7,5,32,0.571429,0.285714,0.285714,0.428571,0.000000,1,...,0,0,0,0,0,0,0,0,0,0
1,test_1,4,3,54,0.250000,0.000000,0.000000,0.750000,0.250000,0,...,0,0,0,0,0,0,0,0,0,0
2,test_2,4,2,8,0.000000,0.000000,0.600000,0.400000,0.000000,0,...,0,0,0,0,0,0,0,0,0,0
3,test_3,28,16,16,0.031250,0.062500,0.468750,0.343750,0.125000,3,...,0,0,0,0,0,0,0,0,0,0
4,test_4,19,9,29,0.476190,0.380952,0.285714,0.238095,0.095238,6,...,0,0,0,0,0,0,0,0,0,0


# 마무리

In [ ]:
train_ft.shape, test_ft.shape

((14940, 613), (12225, 613))

In [ ]:
train_ft.to_csv(f"{DATA_PATH}train_common_실험.csv",index=False)
test_ft.to_csv(f"{DATA_PATH}test_common_실험.csv",index=False)